# Algorithms and Complexity — Week 2 Assignment: Karatsuba Multiplication
**Name:** Stephanie Wallace  
**Date:** January 30 2026  
**Topic:** Divide-and-Conquer Multiplication (Simple Recursive vs Karatsuba)

In [13]:
def add_strints(x: str, y: str) -> str:
    """
    Add two nonnegative integer strings by converting to int.
    This method can be rewritten as a sum/carry adder for a
    single digit addition, pulling characters from the
    input strings. For simplicity now, we just convert the
    whole string to integer, do the addition, and then
    convert the number back to string.
    """
    return str(int(x) + int(y))


def simple_recursive_multiplication(x: str, y: str) -> str:
    """
    Recursive multiplication for nonnegative integer strings.
    Assumptions:
      - len(x) == len(y)
      - len(x) is a power of two
      - x and y contain only digits
    Uses:
      xy = ac*10^n + (ad+bc)*10^(n/2) + bd
    """
    # Number of digits in x, y
    n = len(x)
    # Base case
    if n == 1:
        return str(int(x) * int(y))
    # Middle of x, y for splitting them in left/right halves
    m = n // 2
    # Divide x, y into left/right halves
    a = x[:m]
    b = x[m:]
    c = y[:m]
    d = y[m:]
    # Compute the partial solution
    ac = simple_recursive_multiplication(a, c)
    ad = simple_recursive_multiplication(a, d)
    bc = simple_recursive_multiplication(b, c)
    bd = simple_recursive_multiplication(b, d)
    # Conquer the partial solutions
    ad_plus_bc = add_strints(ad, bc)
    # Multiply by powers of 10 via appending zeros (string shift).
    term1 = ac + ("0" * n)
    term2 = ad_plus_bc + ("0" * m)
    # Final sum (Using int conversion for addition to keep things simple)
    
    return str(int(term1) + int(term2) + int(bd))


In [14]:
def sub_strints(x: str, y: str) -> str:
    """
    Subtract two nonnegative integer strings by converting to int.
    Assumes int(x) >= int(y).
    """
    return str(int(x) - int(y))


### Why this cell function?
From my readings and observations, the Karatsuba needs subtraction when computing the “the cross term ad+bc”.
Thus, instead of directly computing `ad` and `bc`, Karatsuba uses:
\[
ad + bc = (a+b)(c+d) - ac - bd
\]
So we need a simple way to subtract string-numbers. It seems from our lectures, I made note that we are allowed to convert strings to integers for basic arithmetic, so `sub_strints` keeps that consistent with `add_strints`.

In [15]:
def strip_leading_zeros(s: str) -> str:
    s = s.lstrip("0")
    return s if s else "0"


def next_power_of_two(n: int) -> int:
    p = 1
    while p < n:
        p *= 2
    return p


def karatsuba_multiplication(x: str, y: str) -> str:
    """
    Wrapper Karatsuba that handles:
      - different lengths
      - lengths that are not powers of two
    by padding to the next power of two.
    """

    x = strip_leading_zeros(x)
    y = strip_leading_zeros(y)

    if x == "0" or y == "0":
        return "0"

    n = max(len(x), len(y))
    p = next_power_of_two(n)

    xx = x.zfill(p)
    yy = y.zfill(p)

    return strip_leading_zeros(_karatsuba_core(xx, yy))


def _karatsuba_core(x: str, y: str) -> str:
    """
    Core Karatsuba assuming:
      - len(x) == len(y)
      - len(x) is a power of two
    """

    n = len(x)
    if n == 1:
        return str(int(x) * int(y))

    m = n // 2
    a, b = x[:m], x[m:]
    c, d = y[:m], y[m:]

    # 3 recursive multiplications
    ac = _karatsuba_core(a, c)
    bd = _karatsuba_core(b, d)

    a_plus_b = add_strints(a, b)
    c_plus_d = add_strints(c, d)

    # IMPORTANT: call the WRAPPER here because (a+b) may have extra digits
    ab_cd = karatsuba_multiplication(a_plus_b, c_plus_d)

    # cross term = (a+b)(c+d) - ac - bd
    cross = sub_strints(sub_strints(ab_cd, ac), bd)

    term1 = ac + ("0" * n)     # ac * 10^n  (since n = 2m)
    term2 = cross + ("0" * m)  # cross * 10^m

    return str(int(term1) + int(term2) + int(bd))

### The Karatsuba Implementation
### Debugging / Design note (Why I added a wrapper)
My first Karatsuba attempt sometimes threw an error during testing (`int('')`), which apparently happens when a recursive call ends up receiving an empty string.

I learnt the reasoning behind such is that the Karatsuba computes an extra product \((a+b)(c+d)\). The values \((a+b)\) and \((c+d)\) can have more digits than the original halves (for example, 99 + 99 = 198). That means the next recursive call may no longer satisfy the assumptions “same length” and “power-of-2 length,” so splitting can become inconsistent and eventually produce a `''` slice.

To fix this cleanly, I added a **wrapper** that pads inputs to the next power of two before calling the core recursive function. This keeps the recursion structure valid while still following the rule style from the class report, where we can use `int(...)` for simple addition/subtraction.

In [16]:
tests = [
    ("12", "34"),
    ("99", "99"),
    ("0123", "0456"),
    ("1234", "5678"),
    ("0000", "0000"),
    ("1111", "0001"),
    ("1234567890123456", "9876543210123456"),
    ("12345678901234561234567890123456", "12345678901234561234567890123456"),
]

print("Testing SIMPLE (Test code from the class report):")
for x, y in tests:
    got = simple_recursive_multiplication(x, y)
    want = str(int(x) * int(y))
    print(f"{x} * {y} ok? {got == want}")

print("\nTesting KARATSUBA (wrapper):")
for x, y in tests:
    got = karatsuba_multiplication(x, y)
    want = str(int(x) * int(y))
    print(f"{x} * {y} ok? {got == want}")

Testing SIMPLE (Test code from the class report):
12 * 34 ok? True
99 * 99 ok? True
0123 * 0456 ok? True
1234 * 5678 ok? True
0000 * 0000 ok? True
1111 * 0001 ok? True
1234567890123456 * 9876543210123456 ok? True
12345678901234561234567890123456 * 12345678901234561234567890123456 ok? True

Testing KARATSUBA (wrapper):
12 * 34 ok? True
99 * 99 ok? True
0123 * 0456 ok? True
1234 * 5678 ok? True
0000 * 0000 ok? True
1111 * 0001 ok? True
1234567890123456 * 9876543210123456 ok? True
12345678901234561234567890123456 * 12345678901234561234567890123456 ok? True


### Observations (Correctness tests)
All test cases returned `ok? True` for both methods:

- The `simple_recursive_multiplication` from the class report matches Python integer multiplication.
- My `karatsuba_multiplication` (wrapper + core) also matches Python integer multiplication.

This suggests the Karatsuba implementation is producing correct outputs for the provided test suite, including:
- numbers with leading zeros (e.g., "0123")
- larger multi-digit inputs (including 16-digit and 32-digit examples)

In [17]:
import time
import random
random.seed(0)

def rand_n_digit_number(n: int) -> str:
    # Ensure it's truly n digits (no leading zero)
    first = str(random.randint(1, 9))
    rest = "".join(str(random.randint(0, 9)) for _ in range(n - 1))
    return first + rest

def time_one(func, x, y, repeats=1):
    start = time.perf_counter()
    for _ in range(repeats):
        func(x, y)
    end = time.perf_counter()
    return (end - start) / repeats

### Notes / Timing setup
This cell defines two helper functions used for benchmarking:

- `rand_n_digit_number(n)` generates random n-digit inputs (first digit is not 0).
- `time_one(...)` measures average runtime using `time.perf_counter()` which is appropriate for small performance comparisons.

I use multiple repeats for small n because the runtime can be extremely small and noisy for a single call.

Also, `time` and `random` are used here only for benchmarking and generating test inputs; the multiplication logic is still implemented as required using string-based recursion.

In [18]:
sizes = [4, 8, 16, 32, 64, 128, 256, 512, 1024, 2048]

print("n_digits | repeats | simple(s) | karatsuba(s) | simple/karatsuba")
print("-" * 65)

for n in sizes:
    x = rand_n_digit_number(n)
    y = rand_n_digit_number(n)

    repeats = 30 if n <= 16 else 20 if n <= 32 else 10 if n <= 64 else 6 if n <= 128 else 4 if n <= 256 else 2 if n <= 1024 else 1

    t_simple = time_one(simple_recursive_multiplication, x, y, repeats=repeats)
    t_kara   = time_one(karatsuba_multiplication, x, y, repeats=repeats)

    ratio = t_simple / t_kara if t_kara > 0 else float("inf")
    print(f"{n:6d} | {repeats:7d} | {t_simple:9.6f} | {t_kara:12.6f} | {ratio:16.3f}")


n_digits | repeats | simple(s) | karatsuba(s) | simple/karatsuba
-----------------------------------------------------------------
     4 |      30 |  0.000039 |     0.000154 |            0.254
     8 |      30 |  0.000154 |     0.000271 |            0.571
    16 |      30 |  0.000592 |     0.003067 |            0.193
    32 |      20 |  0.003670 |     0.006038 |            0.608
    64 |      10 |  0.004694 |     0.012086 |            0.388
   128 |       6 |  0.021187 |     0.059345 |            0.357
   256 |       4 |  0.078815 |     0.177715 |            0.443
   512 |       2 |  0.332949 |     0.724777 |            0.459
  1024 |       2 |  1.583675 |     1.906880 |            0.831
  2048 |       1 |  5.285653 |    10.873215 |            0.486


### Observations (based on my timing results)
In my runs, the simple recursive method was consistently faster than my Karatsuba implementation for all tested sizes from 4 digits up to 2048 digits. This is shown by the ratio column (`simple/karatsuba`) being less than 1 for every n.

For example:
- At n = 64 digits: simple ≈ 0.004694s vs Karatsuba ≈ 0.012086s (ratio ≈ 0.388)
- At n = 1024 digits: simple ≈ 1.583675s vs Karatsuba ≈ 1.906880s (ratio ≈ 0.831)
- At n = 2048 digits: simple ≈ 5.285653s vs Karatsuba ≈ 10.873215s (ratio ≈ 0.486)

My interpretation is that although Karatsuba reduces the number of recursive multiplications, my implementation introduces overhead from:
- padding to the next power of two (the wrapper)
- repeated conversions between strings and Python integers during combining steps
- extra additions/subtractions at each recursion level

So for my current code structure (which prioritizes simplicity), the overhead seems to dominate the theoretical savings at these tested sizes.

Overall, my results suggest that the asymptotic advantage of Karatsuba did not appear in this experiment setup, likely due to implementation overhead rather than the underlying algorithmic idea.

## Observations / Discussion

### What I observed from the timing table
In my benchmark, the simple recursive method was faster than my Karatsuba implementation for all tested sizes (4 to 2048 digits).  
This is shown by `simple/karatsuba < 1` for every row in the table.

A few examples from my results:
- At n = 64 digits: simple ≈ 0.004694s vs Karatsuba ≈ 0.012086s (ratio ≈ 0.388)
- At n = 1024 digits: simple ≈ 1.583675s vs Karatsuba ≈ 1.906880s (ratio ≈ 0.831)
- At n = 2048 digits: simple ≈ 5.285653s vs Karatsuba ≈ 10.873215s (ratio ≈ 0.486)

### My interpretation (why this can happen)
Even though Karatsuba reduces the number of recursive multiplications, my implementation adds overhead:
- padding inputs to the next power of two (wrapper call)
- extra additions/subtractions for the cross term
- repeated `int(...)` conversions when combining results

So, in this experiment setup (which prioritizes simplicity and readability), the overhead likely outweighs the theoretical savings.

### How to handle different lengths and non-power-of-2 inputs
The starter code assumes `len(x) == len(y)` and that the length is a power of two.  
To support general inputs, I would:
1. Pad the shorter string with leading zeros so `len(x) == len(y)`.
2. Pad both strings to the next power of two (e.g., 13 → 16).
3. After multiplication, strip leading zeros from the result.

### Optional improvement idea
If I were optimizing further, I would likely add a cutoff (for small n, directly compute `int(x)*int(y)`) and reduce repeated conversions to integers inside recursion. I suspect this would help Karatsuba’s advantage show up at larger n - my personal observation and opinion...
